# Краткое описание

Блокнот собирает тексты из VK, сайтов и Яндекс Отзывов и приводит их к единому формату `docs`. Запуск выполняется через UI в следующей ячейке.  
Исходный код расположен в скрытой ячейке, которую можно раскрыть при необходимости.

## Как получить токены

### VK (SOIKA)
1. Откройте https://dev.vk.com/ и создайте приложение (тип — Standalone).
2. В разделе **Авторизация** получите пользовательский `access_token` с правами `wall`, `groups`, `offline`.
3. Вставьте токен в поле **VK токен** в форме ниже.

### Яндекс Отзывы
1. Перейдите в https://developer.tech.yandex.ru/ и создайте API-ключ для сервиса **Яндекс Отзывы / Business Reviews API** (если доступ ограничен — запросите доступ в кабинете разработчика).
2. Скопируйте ключ и вставьте его в поле **Yandex токен**.
3. Возьмите ID организации из URL Яндекс Карт вида `https://yandex.ru/maps/org/<название>/<ID>/reviews/` или передайте сам ID.


In [ ]:
from __future__ import annotations

import importlib.util
import re
from typing import Any, Dict, List, Optional

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

from collect_texts_core import (
    FetchConfig,
    collect_vk_soika,
    collect_vk_stub,
    collect_websites_stub,
    collect_yandex_reviews,
    collect_yandex_reviews_stub,
    normalize_vk_domains,
    normalize_yandex_org_ids,
    parse_date_safe,
    parse_websites,
    read_links_from_upload,
    standardize_docs,
)

_is_colab = importlib.util.find_spec("google.colab") is not None
if _is_colab:
    from google.colab import output  # type: ignore

    output.enable_custom_widget_manager()


docs: Optional[pd.DataFrame] = None


def build_ui() -> None:
    global docs

    source_dd = widgets.Dropdown(
        options=[("VK (стены групп)", "vk"), ("Сайты", "website"), ("Яндекс Отзывы", "yandex_reviews")],
        value="vk",
        description="Источник:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="420px"),
    )

    input_ta = widgets.Textarea(
        value="",
        placeholder="""Для VK: ссылки на группы или домены через запятую/строку
Для сайтов: URL-ы по строкам
Для Яндекс Отзывов: URL-ы или ID организаций по строкам""",
        description="Ввод вручную:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="820px", height="140px"),
    )

    single_link_txt = widgets.Text(
        value="",
        placeholder="Быстрая вставка одной ссылки",
        description="Одна ссылка:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="560px"),
    )

    links_upload = widgets.FileUpload(
        accept=".xlsx",
        multiple=False,
        description="XLSX со ссылками",
        style={"description_width": "initial"},
    )

    since_txt = widgets.Text(
        value="",
        placeholder="YYYY-MM-DD (необязательно)",
        description="Период с:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="260px"),
    )

    until_txt = widgets.Text(
        value="",
        placeholder="YYYY-MM-DD (необязательно)",
        description="Период по:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="260px"),
    )

    vk_token_txt = widgets.Password(
        value="",
        placeholder="Токен VK (https://dev.vk.com/api/access-token)",
        description="VK токен:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="480px"),
    )

    yandex_token_txt = widgets.Password(
        value="",
        placeholder="API-ключ Яндекс Отзывов (developer.tech.yandex.ru)",
        description="Yandex токен:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="480px"),
    )

    yandex_limit = widgets.IntSlider(
        value=300,
        min=10,
        max=2000,
        step=10,
        description="Лимит отзывов:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="420px"),
    )

    selector_main = widgets.Text(
        value="",
        placeholder="Для сайтов: article / .post-content / main",
        description="CSS селектор:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="420px"),
    )

    min_chars_main = widgets.IntSlider(
        value=400,
        min=100,
        max=4000,
        step=50,
        description="Мин. длина текста:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="420px"),
    )

    timeout_main = widgets.IntSlider(
        value=20,
        min=5,
        max=120,
        step=5,
        description="Таймаут (сек):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="420px"),
    )

    demo_btn = widgets.Button(description="Загрузить DEMO-корпус", button_style="info")
    run_btn = widgets.Button(description="Запустить сбор", button_style="primary")

    out = widgets.Output()

    website_opts = widgets.Accordion(children=[widgets.VBox([selector_main, min_chars_main, timeout_main])])
    website_opts.set_title(0, "Опции веб-парсера (для источника 'Сайты')")

    vk_opts = widgets.Accordion(children=[vk_token_txt])
    vk_opts.set_title(0, "Параметры VK")

    yandex_opts = widgets.Accordion(children=[widgets.VBox([yandex_token_txt, yandex_limit])])
    yandex_opts.set_title(0, "Параметры Яндекс Отзывов")

    upload_box = widgets.VBox(
        [
            widgets.HBox([links_upload, single_link_txt]),
            input_ta,
        ]
    )

    main_controls = widgets.VBox(
        [
            source_dd,
            upload_box,
            widgets.HBox([since_txt, until_txt]),
            vk_opts,
            yandex_opts,
            website_opts,
            widgets.HBox([demo_btn, run_btn]),
            out,
        ]
    )

    def get_inputs() -> Dict[str, Any]:
        src = source_dd.value
        manual_raw = input_ta.value.strip()
        single = single_link_txt.value.strip()
        since = parse_date_safe(since_txt.value)
        until = parse_date_safe(until_txt.value)

        manual_items: List[str]
        if src == "vk":
            manual_items = [x.strip() for x in re.split(r"[
,;]+", manual_raw) if x.strip()]
        else:
            manual_items = [x.strip() for x in manual_raw.splitlines() if x.strip()]

        items = manual_items
        if single:
            items.append(single)
        items.extend(read_links_from_upload(links_upload))

        return {"source": src, "items": items, "since": since, "until": until}

    def load_demo(_=None) -> None:
        global docs
        with out:
            out.clear_output()
            demo = pd.concat(
                [
                    collect_vk_stub(["demo_group"], None, None),
                    collect_websites_stub(
                        ["https://example.com/article"],
                        selector_main.value or None,
                        min_chars_main.value,
                        timeout_main.value,
                    ),
                    collect_yandex_reviews_stub(["https://example.com/reviews"]),
                ],
                ignore_index=True,
            )
            docs = standardize_docs(demo)
            display(Markdown("✅ Загружен DEMO-корпус. Ниже — первые строки `docs`."))
            display(docs.head(10))

    def run_pipeline(_=None) -> None:
        global docs
        cfg = get_inputs()
        src = cfg["source"]
        items = cfg["items"]

        with out:
            out.clear_output()
            display(
                Markdown(
                    f"**Источник:** `{src}`  
"
                    + f"**Элементы ввода:** {len(items)}  
"
                    + f"**Период:** {cfg['since']} — {cfg['until']}"
                )
            )

            try:
                if src == "vk":
                    token = vk_token_txt.value.strip()
                    if not token:
                        raise ValueError("Для VK укажи токен пользователя в отдельном поле.")
                    groups = normalize_vk_domains(items)
                    if not groups:
                        raise ValueError("Список групп VK пуст. Добавь домены или ссылки на группы.")
                    df = collect_vk_soika(groups, token, cfg["since"])
                elif src == "website":
                    if not items:
                        raise ValueError("Список URL пуст. Добавь ссылки или загрузи XLSX.")
                    df = parse_websites(
                        items,
                        selector=selector_main.value or None,
                        min_chars=int(min_chars_main.value),
                        cfg=FetchConfig(timeout=int(timeout_main.value)),
                    )
                else:
                    if not items:
                        raise ValueError("Список URL пуст. Добавь ссылки или загрузи XLSX.")
                    token = yandex_token_txt.value.strip()
                    if not token:
                        raise ValueError("Для Яндекс Отзывов укажи API-ключ в поле 'Yandex токен'.")
                    org_ids = normalize_yandex_org_ids(items)
                    if not org_ids:
                        raise ValueError(
                            "Не удалось извлечь ID организаций. Передай ссылки из Яндекс Карт или числовые ID."
                        )
                    df = collect_yandex_reviews(org_ids, token, limit=int(yandex_limit.value))

                docs = standardize_docs(df)
                display(Markdown("✅ Получена таблица `docs` (в формате для дальнейших разделов)."))
                display(docs)
            except Exception as e:
                display(Markdown(f"❌ Ошибка: `{e}`"))
                return

    demo_btn.on_click(load_demo)
    run_btn.on_click(run_pipeline)

    display(main_controls)


In [ ]:
build_ui()